In [ ]:
import sys
from pathlib import Path

import pandas as pd
import numpy as np
import geopandas as gpd
import gtfs_kit as gk
import shapely
import shapely.geometry as sg
import geo_kit as geo
import folium as fl

sys.path.append('../')

import make_gtfs as mg


DATA_DIR = Path('../data')

%load_ext autoreload
%autoreload 2


In [ ]:
path = DATA_DIR / 'auckland'
pfeed = mg.read_protofeed(path)
display(pfeed)



In [ ]:
speed_zones = gpd.read_file(DATA_DIR / "auckland/speed_zones.geojson")
speed_zones = pfeed.speed_zones
# speed_zones["route_type"] = 3
# speed_zones["zone_id"] = [str(i) for i in range(speed_zones.shape[0])]
display(speed_zones)

for route_type, group in speed_zones.groupby("route_type"):
    for zone_id, g in group.groupby("zone_id"):
        other = group.loc[lambda x: x.zone_id != zone_id]
        if other.overlaps(g.geometry.iat[0]).any():
            raise ValueError(
                f"Zones must be pairwise disjoint within each route type; "
                f"failure with route type {route_type}"
            )



In [ ]:
pd.api.types.is_numeric_dtype?

In [ ]:
m = geo.plot(
    pfeed.speed_zones.loc[lambda x: x.route_type == 3], 
    color_by="zone_id", 
    add_layer_control=False,
)
m

In [ ]:
def calculate_shape_point_speeds(pfeed, shapes, *, use_utm: bool=False) -> gpd.GeoDataFrame:
    """
    """
    # Get UTM CRS to compute distances in metres
    lat, lon = shapes[["shape_pt_lat", "shape_pt_lon"]].values[0]
    utm_crs = gk.get_utm_crs(lat, lon)
    
    # Build shape points
    def compute_dists(group):
        p2 = group.geometry.shift(1)
        group["dist"] = group.geometry.distance(p2, align=False)
        group["shape_dist_traveled"] = group.dist.fillna(0).cumsum()
        return group

    shape_points = (
        gpd.GeoDataFrame(
            shapes,
            geometry=gpd.points_from_xy(shapes.shape_pt_lon, shapes.shape_pt_lat),
            crs=mg.WGS84,
        )
        .to_crs(utm_crs)
        .sort_values(["shape_id", "shape_pt_sequence"])
        .groupby("shape_id")
        .apply(compute_dists)
        .drop(["dist", "shape_pt_lat", "shape_pt_lon"], axis="columns")
    )
    
    # Get points where shapes intersect speed zone boundaries
    speed_zones = pfeed.speed_zones.to_crs(utm_crs)
    shapes_g = (
        gk.geometrize_shapes_0(shapes)
        .to_crs(utm_crs)
        .assign(boundary_points=lambda x: x.intersection(speed_zones.boundary))
    )

    # Assign distances to those boundary points
    rows = []
    for shape_id, group in shapes_g.groupby("shape_id"):
        bd = group.boundary_points.iat[0]
        if not bd.is_empty:
            for point in bd.geoms:
                dist = group.geometry.iat[0].project(point)
                rows.append([shape_id, dist, point])

    boundary_points = gpd.GeoDataFrame(
        rows, 
        columns=["shape_id", "shape_dist_traveled", "geometry"],
        crs=utm_crs,
    )

    # Concatenate the boundary points with the shape points and assign speeds
    g = (
        pd.concat([
            shape_points,
            boundary_points.assign(shape_pt_sequence=-1),
        ])
        .sort_values(["shape_id", "shape_dist_traveled", "shape_pt_sequence"])
        .sjoin(speed_zones)
        .sort_values(["shape_id", "shape_dist_traveled"])
        .drop("index_right", axis="columns")
    )

    if not use_utm:
        g = g.to_crs(mg.WGS84)

    return g


In [ ]:
shape_point_speeds = calculate_shape_point_speeds(pfeed, shapes)
display(shape_point_speeds.head(30))

# shapes = mg.build_shapes(pfeed)
# shapes_g = gk.geometrize_shapes_0(shapes, use_utm=True)

# stops = mg.build_stops(pfeed)
# stops_g = gk.geometrize_stops_0(stops, use_utm=True)




In [ ]:
g = (
    pd.concat([gk.geometrize_shapes_0(shapes), pfeed.speed_zones])
    .assign(uid=lambda x: x.zone_id.fillna(x.shape_id))
)          
style_fn = lambda feature: {
    "weight": 2,
    "fillOpacity": 0.3,
    "opacity": 1,
    "color": feature["properties"]["_color"],
    "fillColor": feature["properties"]["_color"],
}
geo.plot(g, group_by="uid", color_by="uid", style_function=style_fn)


In [ ]:
shape_id = shapes.shape_id.iat[0]

linestring = shapes_g.loc[lambda x: x.shape_id == shape_id, "geometry"]
stops_g_nearby = mg.get_nearby_stops(stops_g, linestring, "left")
display(stops_g_nearby)

f = (
    stops_g_nearby
    .assign(
        shape_id=shape_id,
        shape_dist_traveled=lambda x: x.apply(lambda row: linestring.project(row.geometry), axis=1),
    )
    .filter(["stop_id", "shape_id", "shape_dist_traveled"])
    .sort_values("shape_dist_traveled")
)
display(f)

ff = (
    pd.concat([f, shape_point_speeds.loc[lambda x: x.shape_id == shape_id]])
    .sort_values("shape_dist_traveled")
    .drop("geometry", axis="columns")
)

stop_stats = (
    ff
    .assign(
        speed=lambda x: x.speed.bfill(),
        dist_to_next=lambda x: x.shape_dist_traveled.diff().shift(-1).fillna(0),
        weight_to_next=lambda x: x.dist_to_next * x.speed,
        shape_weight_traveled=lambda x: x.weight_to_next.cumsum().shift(1).fillna(0),
    )
)
display(stop_stats)

stop_stats = (
    stop_stats
    .loc[lambda x: x.stop_id.notna()]
    .assign(
        dist_to_next=lambda x: x.shape_dist_traveled.diff().shift(-1).fillna(0),
        weight_to_next=lambda x: x.shape_weight_traveled.diff().shift(-1).fillna(0),
        speed_to_next=lambda x: (x.weight_to_next / x.dist_to_next).fillna(0),
    )
#         duration_to_next=lambda x: x.dist_to_next * x.speed_to_next * (1000 / 3600),
#         duration=lambda x: x.duration_to_next.shift(1).fillna(0).cumsum(),
)
display(stop_stats)

In [ ]:
f = (
    feed.routes[['route_id', 'route_short_name']]
    .merge(pfeed.frequencies)
    .merge(pfeed.service_windows)
    .sort_values('route_id')
)
f

In [ ]:
# Map some trips
prefix = 't-rB-weekday_peak_1-07:00:00'
t = (
    feed.trips
    .loc[lambda x: x.trip_id.str.startswith(prefix)]
)
tids = [t.trip_id.iat[0], t.trip_id.iat[-1]]
feed.map_trips(tids, include_arrows=True)
    

In [ ]:
#feed.write("../data/test_write_feed")